<a href="https://colab.research.google.com/github/CnJsTiger/xtts_google_colab/blob/main/XTTS_FT_ipynb_Tigerzhou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset building + XTTS finetuning and inference

#### Running the demo
To start the demo run the first two cells (ignore pip install errors in the first one)

Then click on the link `Running on public URL: ` when the demo is ready.

#### Downloading the results

You can run cell [3] to zip and download default dataset path

You can run cell [4] to zip and download the latest model you trained

### Installing the requirements

In [ ]:
!rm -rf TTS/ # delete repo to be able to reinstall if needed
!git clone --branch xtts_demo -q https://github.com/coqui-ai/TTS.git
!pip install --use-deprecated=legacy-resolver -q -e TTS
!pip install --use-deprecated=legacy-resolver -q -r TTS/TTS/demos/xtts_ft_demo/requirements.txt
!pip install -q typing_extensions==4.8 numpy==1.26.2

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 M

### Running the gradio UI

In [ ]:
!python TTS/TTS/demos/xtts_ft_demo/xtts_demo.py --batch_size 2 --num_epochs 6

2023-12-11 12:48:50.426905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 12:48:50.426976: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 12:48:50.427026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 12:48:51.563649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running on local URL:  http://0.0.0.0:5003
Running on public URL: https://7537d2d96d24c8cb44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spa

### Downloading the dataset

In [ ]:
from google.colab import files

!zip -q -r dataset.zip /tmp/xtts_ft/dataset
files.download('dataset.zip')

### Downloading the model

In [ ]:
from google.colab import files
import os
import glob
import torch

def find_latest_best_model(folder_path):
    search_path = os.path.join(folder_path, '**', 'best_model.pth')
    files = glob.glob(search_path, recursive=True)
    latest_file = max(files, key=os.path.getctime, default=None)
    return latest_file

model_path = find_latest_best_model("/tmp/xtts_ft/run/training/")
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
del checkpoint["optimizer"]
for key in list(checkpoint["model"].keys()):
    if "dvae" in key:
        del checkpoint["model"][key]
torch.save(checkpoint, "model.pth")
model_dir = os.path.dirname(model_path)
files.download(os.path.join(model_dir, 'config.json'))
files.download(os.path.join(model_dir, 'vocab.json'))
files.download('model.pth')

### Copy files to your google drive

The two previous cells are a requirement for this step but it can be much faster

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/XTTS_ft_colab
shutil.copy(os.path.join(model_dir, 'config.json'), "/content/drive/MyDrive/XTTS_ft_colab/config.json")
shutil.copy(os.path.join(model_dir, 'vocab.json'), "/content/drive/MyDrive/XTTS_ft_colab/vocab.json'")
shutil.copy('model.pth', "/content/drive/MyDrive/XTTS_ft_colab/model.pth")
shutil.copy('dataset.zip', "/content/drive/MyDrive/XTTS_ft_colab/dataset.zip")